# Generating dataset for training and validation

## Selecting images related to swimming

In [86]:
import os
import sys
import shutil
import copy
from pathlib import Path

import scipy.io
# import numpy as np
# from tqdm import tqdm
# from adjustText import adjust_text
# from matplotlib import pyplot as plt
# from matplotlib.patches import Circle

In [2]:
root = Path(os.getcwd())
sys.path.append(root)

dataset_name = ['mpii']
dataset_path = list(map(lambda x: os.path.join(root, 'data', x), dataset_name))

In [3]:
# Load the mat file.
matlab_mpii = scipy.io.loadmat(os.path.join(dataset_path[0], 'joints.mat'), struct_as_record=False)['RELEASE'][0, 0]
num_images = annotation_mpii = matlab_mpii.__dict__['annolist'][0].shape[0]

In [4]:
matlab_mpii.__dict__

{'_fieldnames': ['annolist',
  'img_train',
  'version',
  'single_person',
  'act',
  'video_list'],
 'annolist': array([[<scipy.io.matlab._mio5_params.mat_struct object at 0x7fac12618190>,
         ...,
       dtype=object),
 'img_train': array([[0, 0, 0, ..., 1, 0, 0]], dtype=uint8),
 'version': array(['12'], dtype='<U2'),
 'single_person': array([[array([[1]], dtype=uint8)],
        [array([], shape=(0, 1), dtype=uint8)],
        [array([[1]], dtype=uint8)],
        ...,
        [array([[5]], dtype=uint8)],
        [array([], shape=(0, 1), dtype=uint8)],
        [array([], shape=(0, 1), dtype=uint8)]], dtype=object),
 'act': array([[<scipy.io.matlab._mio5_params.mat_struct object at 0x7fabe5163520>],
        ...,
       dtype=object),
 'video_list': array([[array(['-08Vnk8XONY'], dtype='<U11'),
         array(['-0YdHqlKV8w'], dtype='<U11'),
         array(['-1nf9pRuuZw'], dtype='<U11'), ...,
         array(['zvMWkSAcSVc'], dtype='<U11'),
         array(['zwqQrtD2L84'], dtype='<U11'

### Putting the [official docs](http://human-pose.mpi-inf.mpg.de/#download) here for reference...

#### Annotation description 
Annotations are stored in a matlab structure `RELEASE` having following fields

- `.annolist(imgidx)` - annotations for image `imgidx`
  - `.image.name` - image filename
  - `.annorect(ridx)` - body annotations for a person `ridx`
		  - `.x1, .y1, .x2, .y2` - coordinates of the head rectangle
		  - `.scale` - person scale w.r.t. 200 px height
		  - `.objpos` - rough human position in the image
		  - `.annopoints.point` - person-centric body joint annotations
		    - `.x, .y` - coordinates of a joint
		    - `id` - joint id 
[//]: # "(0 - r ankle, 1 - r knee, 2 - r hip, 3 - l hip, 4 - l knee, 5 - l ankle, 6 - pelvis, 7 - thorax, 8 - upper neck, 9 - head top, 10 - r wrist, 10 - r wrist, 12 - r shoulder, 13 - l shoulder, 14 - l elbow, 15 - l wrist)"
		    - `is_visible` - joint visibility
  - `.vidx` - video index in `video_list`
  - `.frame_sec` - image position in video, in seconds
 
- `img_train(imgidx)` - training/testing image assignment 
- `single_person(imgidx)` - contains rectangle id `ridx` of *sufficiently separated* individuals
- `act(imgidx)` - activity/category label for image `imgidx`
  - `act_name` - activity name
  - `cat_name` - category name
  - `act_id` - activity id
- `video_list(videoidx)` - specifies video id as is provided by YouTube. To watch video on youtube go to https://www.youtube.com/watch?v=video_list(videoidx) 


In [59]:
activity = matlab_mpii.__dict__['act'][1223][0].__dict__['act_name'][0]
if 'ball' in activity or 'Ball' in activity:
    print(activity)
#     print(annotation_mpii.__dict__['image'][1223, 0].__dict__['name'][0])
    print(matlab_mpii.__dict__['annolist'][0][1223].__dict__['image'][0, 0].__dict__['name'][0])

therapeutic exercise ball, Fitball exercise
094361592.jpg


In [115]:
swimming_images = []
activities = set()

for i in range(num_images):
    activity = matlab_mpii.__dict__['act'][i][0].__dict__['act_name']
    if activity.shape[0] == 0:
        continue 
    if 'swimming' in activity[0] or 'Swimming' in activity[0]:
        activities.add(activity[0])
        img_name = matlab_mpii.__dict__['annolist'][0][i].__dict__['image'][0, 0].__dict__['name'][0]
        swimming_images.append(img_name)
       
print(activities)
print(len(swimming_images))

{'swimming, backstroke', 'swimming, sidestroke, general', 'swimming, breaststroke, recreational', 'swimming, lake, ocean, river (Taylor Codes 280, 295)', 'swimming, synchronized', 'swimming, general', 'swimming, butterfly, general'}
258


In [79]:
def write_array_to_file(array, filename):
    with open(filename, 'w') as file:
        for item in array:
            file.write(str(item) + '\n')

In [82]:
file_name = 'data/mpii/swimming_images.txt'
write_array_to_file(swimming_images, file_name)

In [94]:
# hmm, some images like 002256858.jpg don't look quite like what we're looking for...

sus_images = []
for i in range(num_images):
    activity = matlab_mpii.__dict__['act'][i][0].__dict__['act_name']
    if activity.shape[0] == 0:
        continue 
    if activity[0] == 'swimming, lake, ocean, river (Taylor Codes 280, 295)':
        img_name = matlab_mpii.__dict__['annolist'][0][i].__dict__['image'][0, 0].__dict__['name'][0]
        sus_images.append(img_name)

print(len(sus_images))

67


After visual inspection, images in two categories: `'swimming, lake, ocean, river (Taylor Codes 280, 295)'` and `'swimming, synchronized'` don't quite fit with our criteria.

In [116]:
activities.remove('swimming, synchronized')
activities.remove('swimming, lake, ocean, river (Taylor Codes 280, 295)')

In [117]:
activities

{'swimming, backstroke',
 'swimming, breaststroke, recreational',
 'swimming, butterfly, general',
 'swimming, general',
 'swimming, sidestroke, general'}

In [118]:
refined_swimming_images = []

for i in range(num_images):
    activity = matlab_mpii.__dict__['act'][i][0].__dict__['act_name']
    if activity.shape[0] == 0:
        continue 
    if activity[0] in activities:
        activities.add(activity[0])
        img_name = matlab_mpii.__dict__['annolist'][0][i].__dict__['image'][0, 0].__dict__['name'][0]
        refined_swimming_images.append(img_name)
       
print(activities)
print(len(refined_swimming_images))

{'swimming, backstroke', 'swimming, sidestroke, general', 'swimming, breaststroke, recreational', 'swimming, general', 'swimming, butterfly, general'}
174


In [92]:
# copying data to own folder
def copy_files(file_names, source_folder, destination_folder):
    
    if not os.path.exists(destination_folder):
        # Create a new directory because it does not exist
        os.makedirs(destination_folder)
    
    for file_name in file_names:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy(source_path, destination_path)
    print('%s images copied over to %s' %(len(file_names), destination_folder))

258 images copied over to data/mpii/swimming_images


In [98]:
def delete_matching_images(source_folder, target_folder):
    source_files = os.listdir(source_folder)
    target_files = os.listdir(target_folder)
    matching_files = set(source_files) & set(target_files)  # Find the common files

    for file_name in matching_files:
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)

In [122]:
! rm -r data/mpii/swimming_images

In [123]:
file_names = refined_swimming_images
source_folder = 'data/mpii/images'
destination_folder = 'data/mpii/swimming_images'

copy_files(refined_swimming_images, source_folder, destination_folder)

174 images copied over to data/mpii/swimming_images


## Annotating data

See if we could calculate the center of mass for a certain image.

four joints to be identifiable in an image: L/R shoulders, L/R hips 


Using a [data augmentation pipeline](https://github.com/KaiserZZK/Visual-Affordance-Learning-for-Robotic-Grasping/tree/master#2c-augumenteddataset-in-trainpy) to increase the training sample size, given that we have quite limited number of images to work with?